Importing Required Libraries 

In [231]:
import pandas as pd
import numpy as np
import pickle 
import seaborn as sns  
import plotly.express as px
from typing import Counter
import scikitplot as skplt
from numpy import where
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from imblearn import under_sampling, over_sampling
from sklearn.decomposition import  PCA
from sklearn.ensemble import IsolationForest
from imblearn.over_sampling import SMOTE, RandomOverSampler
from sklearn.ensemble import  RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report 
from sklearn.model_selection import  cross_val_score,StratifiedKFold

Loading the Dataset

In [232]:
pip install scikit-plot

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\program files\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [233]:
pip install seaborn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\program files\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [234]:
data = pd.read_csv("three_parameters.csv")
df = pd.DataFrame(data)
df

,Landmark_1,Landmark_2,Landmark_3,Class
0,8.944272,8.062258,35.114100,Drowsy
1,8.944272,8.062258,31.780497,Unknown
2,8.062258,8.062258,35.114100,Drowsy
3,9.433981,9.433981,33.105891,Unknown
4,8.062258,8.602325,36.055513,Drowsy
...,...,...,...,...
4053,2.000000,2.000000,12.000000,Unknown
4054,4.000000,3.162278,13.000000,Drowsy
4055,2.236068,3.000000,12.165525,Unknown
4056,10.049876,9.055385,33.015148,Drowsy


In [235]:
df.round(1)

,Landmark_1,Landmark_2,Landmark_3,Class
0,8.9,8.1,35.1,Drowsy
1,8.9,8.1,31.8,Unknown
2,8.1,8.1,35.1,Drowsy
3,9.4,9.4,33.1,Unknown
4,8.1,8.6,36.1,Drowsy
...,...,...,...,...
4053,2.0,2.0,12.0,Unknown
4054,4.0,3.2,13.0,Drowsy
4055,2.2,3.0,12.2,Unknown
4056,10.0,9.1,33.0,Drowsy


Exploratory Data Analysis(EDA)

In [236]:
len(df)

4058

In [237]:
df.describe()

,Landmark_1,Landmark_2,Landmark_3
count,4058.000000,4058.000000,4058.000000
mean,7.492809,7.383439,32.124084
std,1.999815,1.931438,3.298754
min,1.000000,1.000000,10.000000
25%,6.082763,6.082763,30.149627
50%,8.000000,7.280110,32.140317
75%,9.055385,9.000000,34.234486
max,14.142136,14.317821,44.011362


In [238]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4058 entries, 0 to 4057
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Landmark_1  4058 non-null   float64
 1   Landmark_2  4058 non-null   float64
 2   Landmark_3  4058 non-null   float64
 3   Class       4058 non-null   object 
dtypes: float64(3), object(1)
memory usage: 126.9+ KB


In [239]:
df.duplicated().sum()

989

In [240]:
df.isnull().sum()

Landmark_1    0
Landmark_2    0
Landmark_3    0
Class         0
dtype: int64

In [241]:
count = 0
for i in range(len(df)):
    if df['Class'][i] == "Unknown":
        count += 1
count

2029

In [242]:
count = 0
for i in range(len(df)):
    if df['Class'][i] == "Alert":
        count += 1
count

191

In [243]:
len(df['Class'])

4058

In [244]:
data.shape

(4058, 4)

In [245]:
data = data[data['Class'] != 'Unknown']
data.shape
df = data
df.shape

(2029, 4)

In [246]:
numeric_features = ['Landmark_1', 'Landmark_2', 'Landmark_3']

# Plot histograms for each numeric feature
for feature in numeric_features:
    fig = px.histogram(df, x=feature, title=f'Distribution of {feature}')
    fig.update_layout(
        xaxis_title=feature,
        yaxis_title='Frequency',
    )
    fig.show()

In [247]:
df

,Landmark_1,Landmark_2,Landmark_3,Class
0,8.944272,8.062258,35.114100,Drowsy
2,8.062258,8.062258,35.114100,Drowsy
4,8.062258,8.602325,36.055513,Drowsy
6,8.062258,8.062258,35.468296,Drowsy
8,8.602325,7.211103,34.176015,Drowsy
...,...,...,...,...
4048,12.041595,11.045361,33.000000,Alert
4050,3.000000,2.000000,11.045361,Drowsy
4052,2.000000,2.000000,13.038405,Drowsy
4054,4.000000,3.162278,13.000000,Drowsy


In [248]:
fig = px.scatter_3d(df, x='Landmark_1', y='Landmark_2', z='Landmark_3',color='Class')
fig.update_layout(
    title='Drowsy Data set',
    dragmode='select',
    width=600,
    height=600,
    hovermode='closest',
)
fig.show()

In [249]:
fig = px.violin(df, y = 'Landmark_1')
fig.show()

In [250]:
fig = px.scatter_matrix(df)
fig.update_layout(
    title='Drowsy Data set',
    dragmode='select',
    width=600,
    height=600,
    hovermode='closest',
)
fig.show()

In [ ]:
#Oversamopling and UnderSampling 

In [ ]:
X = df.drop(columns=["Class"])
Y = df['Class']

In [ ]:
X.shape

(2029, 3)

In [ ]:
outlier_detect = IsolationForest(contamination=0.05, random_state=42)
outlier = outlier_detect.fit_predict(X)
outlier

array([ 1,  1,  1, ..., -1, -1,  1])

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

#Normalizing theFreatures
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
print(X_train.shape)
print(X_test.shape)

(1420, 3)
(609, 3)


In [ ]:
oversampler = RandomOverSampler(sampling_strategy='auto', random_state=42)
X_train_resampled, Y_train_resampled = oversampler.fit_resample(X_train,Y_train)

smote = SMOTE(random_state=42)
X_resampled, Y_resampled = smote.fit_resample(X, Y)
X_resampled
print(Counter(Y_resampled))

Counter({'Drowsy': 1838, 'Alert': 1838})


In [ ]:
pca = PCA(n_components=2)  
X_pca = pca.fit_transform(X_resampled)

In [ ]:
clf = RandomForestClassifier(random_state=42)
clf

RandomForestClassifier(random_state=42)

In [ ]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30],
}
grid_search = GridSearchCV(clf, param_grid, cv=5)
grid_search.fit(X_train, Y_train)

best_clf = grid_search.best_estimator_

Y_pred = best_clf.predict(X_test)
accuracy = accuracy_score(Y_test, Y_pred)
report = classification_report(Y_test, Y_pred)

print(f"Accuracy: {accuracy}")
print(report)

Accuracy: 0.9688013136288999
              precision    recall  f1-score   support

       Alert       0.94      0.76      0.84        66
      Drowsy       0.97      0.99      0.98       543

    accuracy                           0.97       609
   macro avg       0.96      0.88      0.91       609
weighted avg       0.97      0.97      0.97       609



In [ ]:
counter = Counter(Y)
counter

Counter({'Drowsy': 1838, 'Alert': 191})

In [ ]:
df.shape
Counter(df['Class'])

Counter({'Drowsy': 1838, 'Alert': 191})

In [ ]:
#Cross Validation
K_fold = 5
cv = StratifiedKFold(n_splits=K_fold, shuffle=True, random_state=42)
scores = cross_val_score(clf, X, Y, cv=cv, scoring='accuracy')
for fold, score in enumerate(scores, 1):
    print(f"Fold {fold} - Accuracy: {score:.4f}")
mean_accuracy = scores.mean()
std_accuracy = scores.std()

print(f"Mean Accuracy: {mean_accuracy:.4f}")
print(f"Standard Deviation: {std_accuracy:.4f}")

Fold 1 - Accuracy: 0.9581
Fold 2 - Accuracy: 0.9729
Fold 3 - Accuracy: 0.9704
Fold 4 - Accuracy: 0.9704
Fold 5 - Accuracy: 0.9728
Mean Accuracy: 0.9690
Standard Deviation: 0.0055


In [ ]:
df

,Landmark_1,Landmark_2,Landmark_3,Class
0,8.944272,8.062258,35.114100,Drowsy
2,8.062258,8.062258,35.114100,Drowsy
4,8.062258,8.602325,36.055513,Drowsy
6,8.062258,8.062258,35.468296,Drowsy
8,8.602325,7.211103,34.176015,Drowsy
...,...,...,...,...
4048,12.041595,11.045361,33.000000,Alert
4050,3.000000,2.000000,11.045361,Drowsy
4052,2.000000,2.000000,13.038405,Drowsy
4054,4.000000,3.162278,13.000000,Drowsy


In [ ]:
filename = "random_forest_model.pkl"
with open(filename, 'wb') as file:
    pickle.dump(best_clf,file)
input_landmarks = [[3.0, 2.0, 11.04]]  
predictions = best_clf.predict(input_landmarks)
print("Predicted Class:", predictions[0])

Predicted Class: Drowsy
